In [1]:
import numpy as np
import elasticity as ela
from dolfinx import fem, mesh, io
import ufl

In [2]:
elpars = ela.ElasticPars()
print(elpars)

ElasticPars(nelx=32, nely=32, nelz=32, lmd = 0.6, mu = 0.4, E=1.142857142857143, mu=0.4)


In [3]:
fea = ela.Elasticity(elpars)

3 dimensional problem, and mesh size of 32 x 32 x 32 is created


In [4]:
fea.set_boundary_condition()

currently supports left clamp problem only
currently supports point load only


In [5]:
# setup density
density = fem.Function(fea.D0) # if C1, vector per node: power is not defined 
density.x.array[:] = 1.0

In [6]:
fea.setup_problem(density=density, penal=3.)

In [7]:
# this automatically calls solve(), which is a wrapper of PETSc solver: parallel
fea.solve_problem()

In [8]:
# pyvista visualization
import pyvista 
import dolfinx.plot as plot

# ref: https://docs.fenicsproject.org/dolfinx/main/python/demos/demo_pyvista.html
# If environment variable PYVISTA_OFF_SCREEN is set to true save a png
# otherwise create interactive plot
if pyvista.OFF_SCREEN:
    pyvista.start_xvfb(wait=0.1)

# Set some global options for all plots
transparent = False
figsize = 800
pyvista.rcParams["background"] = [0.5, 0.5, 0.5]

In [9]:
# Create VTK mesh
cells, types, x = plot.create_vtk_mesh(fea.msh)
grid = pyvista.UnstructuredGrid(cells, types, x)

# # attach the values of u
# grid.point_data["u"] = fea.u_sol.x.array
# grid.set_active_scalars("displacement")

# Create point cloud of vertices, and add the vertex values to the cloud
print((fea.u_sol.x.array.reshape(x.shape[0], fea.dim)).shape)
grid.point_data["u"] = fea.u_sol.x.array.reshape(x.shape[0], fea.dim)
glyphs = grid.glyph(orient="u", factor=0.1)

 # We would also like to visualize the underlying mesh and obtain
# that as we have done previously
num_cells = fea.msh.topology.index_map(fea.msh.topology.dim).size_local
cell_entities = np.arange(num_cells, dtype=np.int32)
cells, types, x = plot.create_vtk_mesh(fea.msh, entities=cell_entities)
org_grid = pyvista.UnstructuredGrid(cells, types, x)


(35937, 3)


In [13]:
pyvista.Report(gpu=True)

--------------------------------------------------------------------------------
  Date: Sat May 13 12:58:46 2023 KST

                OS : Linux
            CPU(s) : 32
           Machine : x86_64
      Architecture : 64bit
               RAM : 62.5 GiB
       Environment : Jupyter
       File system : ext4
       GPU Details : error
  MathText Support : True

  Python 3.10.11 | packaged by conda-forge | (main, May 10 2023, 18:58:44)
  [GCC 11.3.0]

           pyvista : 0.39.0
               vtk : 9.2.6
             numpy : 1.24.3
        matplotlib : 3.7.1
            scooby : 0.7.2
             pooch : v1.7.0
           IPython : 8.13.2
             scipy : 1.10.1
      nest_asyncio : 1.5.6
--------------------------------------------------------------------------------

In [11]:
# We create a pyvista plotter
# plotter = pyvista.Plotter()
# plotter.add_text("Mesh and corresponding vectors",
#                     position="upper_edge", font_size=14, color="black")

# # We add in the glyphs corresponding to the plotter
# plotter.add_mesh(org_grid, color="white", style="wireframe", line_width=5)
# plotter.add_mesh(glyphs)

# # Save as png if we are using a container with no rendering
# if True: #pyvista.OFF_SCREEN:
#     plotter.screenshot("3D_wireframe_with_vectors.png", transparent_background=transparent,
#                         window_size=[figsize, figsize])
# else:
#     plotter.show()


In [12]:
# from petsc4py import PETSc

# # We visualize the data
# plotter = pyvista.Plotter()
# # plotter.add_text("Second-order (P2) discontinuous elements",
#                     # position="upper_edge", font_size=14, color="black")
# sargs = dict(height=0.1, width=0.8, vertical=False, position_x=0.1, position_y=0, color="black")
# plotter.add_mesh(grid, show_edges=False, scalar_bar_args=sargs, line_width=0)
# plotter.add_mesh(org_grid, color="white", style="wireframe", line_width=5)
# plotter.add_mesh(grid.copy(), style="points", point_size=15, render_points_as_spheres=True, line_width=0)
# plotter.view_xy()
# if pyvista.OFF_SCREEN:
#     plotter.screenshot(f"DG_{PETSc.MPI.COMM_WORLD.rank}.png",
#                         transparent_background=transparent, window_size=[figsize, figsize])
# else:
#     plotter.show()